### Importing All required libraries

In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline

import pickle

###  Reading audio dataset CSV

In [2]:
df = pd.read_csv('audio_dataset.csv')

### Seperating Feature and Target

In [3]:
X = df.iloc[:,0:-1].values # It is the feature
y= df.iloc[:,-1].values    # It is the target

### Spliting feature and target into training and test dataset

In [4]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2, random_state= 42)

### Creating model with Random Forest Classifier

In [5]:
pipeline_Rand = Pipeline([('standard_scaler',StandardScaler()), ('classifier',RandomForestClassifier(n_estimators =50, class_weight= 'balanced'))])
pipeline_Rand.fit(X_train,y_train)

pipeline_Rand.score(X_test, y_test)

0.7341772151898734

### Classification Report To check accuracy of our model

In [6]:
print(classification_report(y_test, pipeline_Rand.predict(X_test)))

                 precision    recall  f1-score   support

   fan_abnormal       0.46      0.42      0.44        62
     fan_normal       0.62      0.73      0.67       101
  pump_abnormal       1.00      0.33      0.50        18
    pump_normal       0.88      1.00      0.93        14
slider_abnormal       0.98      1.00      0.99        58
  slider_normal       0.81      0.88      0.84        99
 valve_abnormal       0.69      0.38      0.49        24
   valve_normal       0.89      0.84      0.86        19

       accuracy                           0.73       395
      macro avg       0.79      0.70      0.72       395
   weighted avg       0.74      0.73      0.72       395



In [7]:
confusion_matrix(y_test, pipeline_Rand.predict(X_test))

array([[26, 32,  0,  0,  0,  3,  1,  0],
       [23, 74,  0,  0,  1,  3,  0,  0],
       [ 3,  5,  6,  2,  0,  2,  0,  0],
       [ 0,  0,  0, 14,  0,  0,  0,  0],
       [ 0,  0,  0,  0, 58,  0,  0,  0],
       [ 3,  8,  0,  0,  0, 87,  1,  0],
       [ 1,  0,  0,  0,  0, 12,  9,  2],
       [ 0,  0,  0,  0,  0,  1,  2, 16]])

### Saving model to file

In [8]:
pickle.dump(pipeline_Rand, open('model.pkl', 'wb'))

### Validate the model for different Abnormal dataset

In [9]:
validate_df = pd.read_csv('validation_dataset.csv', header= None)

X_valid = validate_df.iloc[:,0:-1].values
y_valid= validate_df.iloc[:,-1].values

In [10]:
y_valid = np.where((y_valid == 'data_fan'), 'fan_abnormal',y_valid)
y_valid = np.where((y_valid == 'data_pumps'), 'pump_abnormal',y_valid)
y_valid = np.where((y_valid == 'data_slider'), 'slider_abnormal',y_valid)
y_valid = np.where((y_valid == 'data_valve'), 'valve_abnormal',y_valid)

### Load the model for validation 

In [11]:
loaded_model = pickle.load(open('model.pkl', 'rb'))
result = loaded_model.score(X_test, y_test)
print(result)

0.7341772151898734


### Classification report with validation dataset

In [12]:
print(classification_report(y_valid, loaded_model.predict(X_valid)))

                 precision    recall  f1-score   support

   fan_abnormal       1.00      0.36      0.53       100
     fan_normal       0.00      0.00      0.00         0
  pump_abnormal       1.00      1.00      1.00        50
slider_abnormal       1.00      1.00      1.00       100
  slider_normal       0.00      0.00      0.00         0
 valve_abnormal       0.98      0.88      0.93        50

       accuracy                           0.77       300
      macro avg       0.66      0.54      0.58       300
   weighted avg       1.00      0.77      0.83       300



/home/admin1/anaconda3/envs/monitor_env/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/admin1/anaconda3/envs/monitor_env/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/admin1/anaconda3/envs/monitor_env/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [13]:
print(confusion_matrix(y_valid,loaded_model.predict(X_valid)))

[[ 36  38   0   0  25   1]
 [  0   0   0   0   0   0]
 [  0   0  50   0   0   0]
 [  0   0   0 100   0   0]
 [  0   0   0   0   0   0]
 [  0   0   0   0   6  44]]
